# Compute the path of each link in the dataset

## The dataset

In the DB, we have around 100 000 vectors of 1536 dimensions. Each vector represents the embeddings of a popular (> 100 votes) link on Hacker News. The vectors are computed using an OpenAI model.

## The problem

The goal is to compute the path for each link. Therefore, we would have a walkable categorised graph of links. Perfect to find links related to a subject.

## The solution

We use a clustering algorithm to group the vectors. Thanks to scikit-learn, we can use the [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) algorithm. We first divide the vectors into 8 clusters. Then, we divide each cluster into 8 clusters. We repeat this 6 times (because log8(100 000) ~= 5.5365).

To represent the data in PostgreSQL, we use a table with the following columns:
- id: the id of the post on Hacker News
- embedding: the vector of 1536 dimensions
- path: an array of integers representing the path (e.g [0, 5, 2] means the 0/5/2 path)


In [3]:
import psycopg2
from json import loads
import numpy as np
from sklearn.cluster import KMeans
from itertools import product

# Connect to the database
conn = psycopg2.connect("dbname=hn user=julien")

# Open a cursor to perform database operations
cur = conn.cursor()

N = 8


In [68]:
conn.rollback()

In [6]:
def getClusterForPoint(points: np.array, N=8) -> np.ndarray:
    """
    Return the cluster for a list of points
    """
    kmeans = KMeans(n_clusters=N, random_state=0, n_init=10).fit(points)
    return kmeans.labels_


def getPointsFromDatabase(path: tuple[int]) -> (list[int], np.ndarray):
    """
    Return a list of ids and a list of points from the database in a path.

    :param path: the path to fetch (e.g. (0, 2, 1))

    :return: a list of ids and a list of points
    """
    print("Fetching points from the database for depth {}".format(path))
    # Execute a query
    query = "SELECT id, embedding FROM hn_embeddings WHERE embedding IS NOT NULL AND path[:{}] = ARRAY[{}]".format(
                    len(path),
                    ",".join(map(str, path))
                )
    
    cur.execute(query)


    # Retrieve query results
    results = cur.fetchall()
    # (1, '[0.1, 0.2, 0.3, 0.4, 0.5]')
    print("Fetched {} points from the database".format(len(results)))

    ids = []
    points = []
    for record in results:
        ids.append(record[0])
        points.append(loads(record[1]))

    print("Finished loading points in the memory")

    return ids, np.array(points)


def setPathInDatabase(id: int, depth: int, number: int) -> None:
    """
    Set the path for a given ID and depth in the database

    :param id: the ID of the directory
    :param depth: the depth of the directory
    :param number: the number of the directory
    """
    # We use depth+1 because the path is 0-indexed but the database is 1-indexed
    cur.execute("UPDATE hn_embeddings SET path[{}] = {} WHERE id = {}".format(
        depth+1, number, id))
    conn.commit()


def getAllPointsFromDatabase() -> (list[int], np.ndarray):
    """
    Return a list of ids and a list of points from the database

    :return: a list of ids and a list of points
    """
    # Execute a query
    cur.execute(
        "SELECT id, embedding FROM hn_embeddings WHERE embedding IS NOT NULL")

    # Retrieve query results
    results = cur.fetchall()
    # (1, '[0.1, 0.2, 0.3, 0.4, 0.5]')

    print("Fetched {} points from the database".format(len(results)))

    ids = []
    points = []
    for record in results:
        ids.append(record[0])
        points.append(loads(record[1]))

    print("Finished loading points in the memory")

    return ids, np.array(points)


def computeRootPath() -> None:
    """
    Compute the root path for all the points in the database
    """
    print("Computing root path for all points in the database")
    ids, points = getAllPointsFromDatabase()

    print("Computing clusters")
    labels = getClusterForPoint(points, N=8)

    print("Setting path in the database")
    for i, label in enumerate(labels):
        setPathInDatabase(ids[i], 0, label)
        print("\r{}/{}".format(i, len(labels)), end="")

    print("\n\n")


def computePath(depth: int) -> None:
    """
    Compute the path for all the points in the database for a given depth
    To do so, get all the permutations of the previous depth and compute 8 clusters
    for each of them

    :param depth: the depth to compute
    """

    numbers = [i for i in range(N)]
    permutations = product(numbers, repeat=depth)
    permutations = list(permutations)

    print("{} permutations for depth {}".format(len(permutations), depth))

    for permutation in permutations:
        # Get the points for the permutation
        print("Computing path for permutation {}".format(permutation))
        ids, points = getPointsFromDatabase(permutation)

        # Return if the number of points is too low
        if len(points) <= 8:
            print("Not enough points for permutation {}".format(permutation))
            continue

        # Compute the clusters
        print("Computing clusters")
        labels = getClusterForPoint(points, N=8)

        # Set the path in the database
        print("Setting path in the database")
        for i, label in enumerate(labels):
            setPathInDatabase(ids[i], depth, label)
            print("\r{}/{}".format(i, len(labels)), end="")

        print("\n\n")
        


        

## Compute the path

We first compute the path for the first level of clusters. Then, we compute the path for the second level of clusters. We repeat this for 6 levels.


In [7]:
computeRootPath()
for depth in range(1, 4):
    computePath(depth)

Computing root path for all points in the database
Fetched 118960 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
118959/118960


8 permutations for depth 1
Computing path for permutation (0,)
Fetching points from the database for depth (0,)
Fetched 11291 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
11290/11291


Computing path for permutation (1,)
Fetching points from the database for depth (1,)
Fetched 13078 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
13077/13078


Computing path for permutation (2,)
Fetching points from the database for depth (2,)
Fetched 15137 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
15136/15137


Computing path for permutation (3,)
Fetching points from the database for depth (3,)
Fetched 18394 points fr

/Users/julien/Code/alexandria-library/.venv/lib/python3.10/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Setting path in the database
215/216


Computing path for permutation (2, 4)
Fetching points from the database for depth (2, 4)
Fetched 2826 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
2825/2826


Computing path for permutation (2, 5)
Fetching points from the database for depth (2, 5)
Fetched 2917 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
2916/2917


Computing path for permutation (2, 6)
Fetching points from the database for depth (2, 6)
Fetched 1524 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
1523/1524


Computing path for permutation (2, 7)
Fetching points from the database for depth (2, 7)
Fetched 858 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
857/858


Computing path for permutation (3, 0)
Fetching points from the d

/Users/julien/Code/alexandria-library/.venv/lib/python3.10/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


20/21


Computing path for permutation (1, 1, 7)
Fetching points from the database for depth (1, 1, 7)
Fetched 106 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
105/106


Computing path for permutation (1, 2, 0)
Fetching points from the database for depth (1, 2, 0)
Fetched 301 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
300/301


Computing path for permutation (1, 2, 1)
Fetching points from the database for depth (1, 2, 1)
Fetched 331 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
330/331


Computing path for permutation (1, 2, 2)
Fetching points from the database for depth (1, 2, 2)
Fetched 186 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
185/186


Computing path for permutation (1, 2, 3)
Fetching points from the database for d

...

Setting path in the database
15/16


Computing path for permutation (7, 2, 6)
Fetching points from the database for depth (7, 2, 6)
Fetched 241 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
240/241


Computing path for permutation (7, 2, 7)
Fetching points from the database for depth (7, 2, 7)
Fetched 124 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
123/124


Computing path for permutation (7, 3, 0)
Fetching points from the database for depth (7, 3, 0)
Fetched 349 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
348/349


Computing path for permutation (7, 3, 1)
Fetching points from the database for depth (7, 3, 1)
Fetched 40 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
39/40


Computing path for permutation (7, 3, 2)
Fetching poin

/Users/julien/Code/alexandria-library/.venv/lib/python3.10/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Setting path in the database
24/25


Computing path for permutation (7, 6, 1)
Fetching points from the database for depth (7, 6, 1)
Fetched 300 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
299/300


Computing path for permutation (7, 6, 2)
Fetching points from the database for depth (7, 6, 2)
Fetched 231 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
230/231


Computing path for permutation (7, 6, 3)
Fetching points from the database for depth (7, 6, 3)
Fetched 226 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
225/226


Computing path for permutation (7, 6, 4)
Fetching points from the database for depth (7, 6, 4)
Fetched 378 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
377/378


Computing path for permutation (7, 6, 5)
Fetching p

/Users/julien/Code/alexandria-library/.venv/lib/python3.10/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Fetched 161 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
160/161


Computing path for permutation (7, 7, 4)
Fetching points from the database for depth (7, 7, 4)
Fetched 167 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
166/167


Computing path for permutation (7, 7, 5)
Fetching points from the database for depth (7, 7, 5)
Fetched 284 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
283/284


Computing path for permutation (7, 7, 6)
Fetching points from the database for depth (7, 7, 6)
Fetched 314 points from the database
Finished loading points in the memory
Computing clusters
Setting path in the database
313/314


Computing path for permutation (7, 7, 7)
Fetching points from the database for depth (7, 7, 7)
Fetched 145 points from the database
Finished loading points in the memory
Computing clu